# Summarizing metrics from disturbed vertical lines

Leanne Friedrich 11/15/22

## initialization

In [ ]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED = False

In [ ]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
from py.tools.config import cfg
import py.tools.logs as logs
from plainIm import plainIm
import matplotlib.pyplot as plt
import py.metric_plots as mp
import py.metrics_disturb as me
import regression as rg
import matplotlib.cm as cm
import py.file_handling as fh
LOGGERDEFINED = logs.openLog('summary_vert.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=False)

## analyzing data

In [ ]:
me.vertDisturbSummaries(os.path.join(cfg.path.server, 'singleDisturb'), os.path.join(cfg.path.fig, 'singleDisturb'), overwrite=False)

In [ ]:
ms = me.metricDisturbSummary(os.path.join(cfg.path.fig, 'singleDisturb', 'vertDisturbSummaries.csv')
                             , diag=True)

In [ ]:
def waterOilPlots(yvar0:str, yideal:float, mode:str='ppt', export:bool=False, top:bool=True):
    if top:
        tlist = ['top', 'bot']
    else:
        tlist = ['bot']
    if mode=='ppt':
        figsize = (9,6+2*(len(tlist)-1))
        fs = 18
        markersize=100
    elif mode=='paper':
        figsize=(6,4)
        fs = 10
        markersize=20
    fig,axs = plt.subplots(len(tlist),2,figsize=figsize, sharey='row', sharex=True)
    if not top:
        axs = [axs]
    ms.ss.sort_values(by=['sigma', 'sup_val'], inplace=True)
    for i,sigma in enumerate(ms.ss.sigma.unique()):
        for j,si in enumerate(tlist):
            yvar = f'{si}_{yvar0}'
            mp.scatterSS(ms, ms.ss[ms.ss.sigma==sigma], 'spacing', yvar, 'sup_val'
                 ,fig=fig, ax=axs[j][i], logx=False, yideal=yideal, xideal=0.875, fontsize=fs, markersize=markersize
                         , legend=False, set_ylabel=False, set_xlabel=(j==len(tlist)-1))
    for axrow in axs:
        for ax in axrow:
            mp.setSquare(ax)
            if ms.ss[yvar].min()>0:
                yy = ms.ss[yvar].max()*0.9
            else:
                yy = ms.ss[yvar].min()*0.9
            ax.text(0.88, yy, 'space filling', color='#555555', fontsize=fs)
            ax.text(1.08, 0.01, 'ideal', color='#555555', fontsize=fs)
    axs[0][0].set_title('Water/water', fontsize=fs)
    axs[0][1].set_title('Oil/water', fontsize=fs)
    if top:
        axs[0][0].set_ylabel('Top')
        axs[1][0].set_ylabel('Bottom')
    fig.supylabel(ms.varSymbol(yvar).replace('bot ',''), fontsize=fs)
    fig.tight_layout()
    if export:
        fig.savefig(os.path.join(cfg.path.fig, 'singleDisturb', 'figures', f'vert_{yvar}.svg'), bbox_inches='tight', dpi=300)

In [ ]:
for s in ['delta_h_n', 'delta_meanT_n', 'delta_xc_n']:
    waterOilPlots(s, 0, top=False)

In [ ]:
ms.ss[(ms.ss.bot_delta_meanT_n<0)&(ms.ss.ink_base=='water')&(ms.ss.spacing==1.25)&(ms.ss.sup_val==2.25)]

In [ ]:
ms.ss[ms.ss.bot_w_dxprint>0.1]

In [ ]:
for s in ['w_dxprint']:
    waterOilPlots(s, 0, export=True, top=True)
for s in ['d_dxf', 'd_space_at', 'd_space_a']:
    waterOilPlots(s, 0, export=True, top=False)

In [ ]:
inspect =ms.ss[ms.ss.bot_w_dxprint>0.1]
inspect

In [ ]:
folder = inspect.iloc[1]['printFolder']
fh.openExplorer(folder)

In [ ]:
me.vertDisturbMeasures(folder, overwrite=True, diag=1, lines=['l0wo', 'l0do', 'l2wo', 'l2do'])

In [ ]:
me.vertDisturbMeasures(folder, overwrite=True, diag=1, lines=['l0w', 'l0d', 'l2w', 'l2d'])

In [ ]:
me.vertDisturbMeasures(folder, overwrite=True, diag=1)
me.vertDisturbSummary(folder, overwrite=True);